In [51]:
import pandas as pd
from tqdm import tqdm
import geopandas as gpd
import pyproj
import requests
from shapely.ops import transform
from shapely.geometry import Point, Polygon, MultiPolygon

In [28]:
data_url = "./make_file/(최종)_서울열선_광진도로.csv"

In [29]:
dong_url = "../데이터_보정/data/BND_ADM_DONG_PG/BND_ADM_DONG_PG.shp"
dong_encoding = "CP949"

In [30]:
data = pd.read_csv(data_url)

In [31]:
# 행정동 경계 데이터 로드
gdf = gpd.read_file(dong_url, encoding=dong_encoding)

In [32]:
data.columns

Index(['도로명', '열선', '도로 종류', '시작점_위도', '시작점_경도', '종료점_위도', '종료점_경도', '중앙점_위도',
       '중앙점_경도', '도로_길이', '도로폭', '도로규모', '행정구역', '고도의_차이', '경사각', '최근접_시설들_거리',
       '최근접_시설의_평균거리', '2019_평균_온도', '2020_평균_온도', '2021_평균_온도', '2022_평균_온도',
       '2023_평균_온도', '2024_평균_온도', '종합_평균_기온', '행정동', '생활인구', '최근접_시설들_최소거리',
       '최근접_시설들_최대거리'],
      dtype='object')

In [33]:
# 좌표 변환 함수 정의
project = pyproj.Transformer.from_crs("EPSG:4326", "EPSG:5186", always_xy=True).transform

In [38]:
def get_admin_dong_name(lon, lat):
    # 위경도 → EPSG:5186 좌표로 변환
    point = transform(project, Point(lon, lat))
    # print("변환된 좌표:", point)

    for _, row in gdf.iterrows():
        geom = row['geometry']
        if isinstance(geom, Polygon):
            if geom.contains(point):
                return row['ADM_NM']
        elif isinstance(geom, MultiPolygon):
            for poly in geom.geoms:
                if poly.contains(point):
                    return row['ADM_NM']
    return '해당 위치의 행정동을 찾을 수 없습니다.'


In [42]:
api_key_url = '../API_KEY/API_KEYS.xlsx'
open_api_key_file = pd.read_excel(api_key_url)
 
print(open_api_key_file['사이트'].value_counts())

KAKAO_API = open_api_key_file[open_api_key_file['사이트'] == 'kakao_api_key_REST_sub'].values[0][1]  

사이트
공공데이터                     1
kakao_api_key_REST        1
kakao_api_key_REST_sub    1
카카오 JS                    1
국토부v-world                1
GOOGLE_MAP                1
행정안전부                     1
Name: count, dtype: int64


In [43]:
def get_gu_from_dong(input_dong_name) :
    dong_name = "서울시 " + input_dong_name
    url = "https://dapi.kakao.com/v2/local/search/keyword.json"
    headers = {"Authorization": f"KakaoAK {KAKAO_API}"}
    params = {"query": dong_name}
    
    response = requests.get(url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API 요청 실패: {response.status_code}, {response.text}")
    
    result = response.json()
    if not result["documents"]:
        raise ValueError("검색 결과 없음")
    
    first_result = result["documents"][0]
    x, y = first_result["x"], first_result["y"]
    
    # 좌표 -> 행정구역 변환
    region_url = "https://dapi.kakao.com/v2/local/geo/coord2regioncode.json"
    params = {"x": x, "y": y}
    response = requests.get(region_url, headers=headers, params=params)
    if response.status_code != 200:
        raise Exception(f"API 요청 실패: {response.status_code}, {response.text}")
    
    region_result = response.json()
    for region in region_result["documents"]:
        if region["region_type"] == "B":  # 법정동(B) 대신 행정동(H) 확인 가능
            return region["region_2depth_name"]  # 구 정보 반환
    
    raise ValueError("구 정보를 찾을 수 없음")


In [55]:
delete_index = []

for i in tqdm(range(len(data))) :
    if data.loc[i]['열선'] == 0 :
        lat = data.loc[i]['중앙점_위도']
        lon = data.loc[i]['중앙점_경도'] 
        result = get_admin_dong_name(lon, lat)
        find_gu = get_gu_from_dong(result) 
        if find_gu != "광진구":
            # print(lat, lon)
            delete_index.append(i)

data.drop(index=delete_index, inplace=True)
data.reset_index(drop=True, inplace=True)


 69%|██████████████████████████▊            | 4948/7212 [14:50<08:52,  4.25it/s]

37.5379313 127.0572334


 69%|██████████████████████████▊            | 4949/7212 [14:50<09:02,  4.17it/s]

37.5379313 127.0572334


 69%|██████████████████████████▊            | 4950/7212 [14:50<09:10,  4.11it/s]

...

 91%|███████████████████████████████████▍   | 6564/7212 [21:05<02:27,  4.40it/s]

37.5716731 127.0803423


 92%|███████████████████████████████████▋   | 6601/7212 [21:13<02:15,  4.52it/s]

37.57119745 127.0843172


100%|███████████████████████████████████████| 7212/7212 [23:32<00:00,  5.11it/s]


In [56]:
delete_index

[4947,
 4948,
 4949,
 4950,
 4951,
 4952,
 4953,
 4954,
 4955,
 4956,
 4957,
 4958,
 4977,
 4980,
 4981,
 5003,
 5004,
 5005,
 5006,
 5007,
 5008,
 5009,
 5010,
 5011,
 5012,
 5013,
 5014,
 5015,
 5122,
 5126,
 5127,
 5173,
 6494,
 6495,
 6496,
 6497,
 6498,
 6499,
 6500,
 6501,
 6502,
 6503,
 6504,
 6505,
 6506,
 6507,
 6508,
 6509,
 6510,
 6511,
 6512,
 6563,
 6600]

In [57]:
len(data)

7159

In [58]:
try :
    data.to_csv(data_url, encoding = "UTF-8", index=False)
    print(f"{data_url} 덮어쓰기 완료.")
except OSError as e:
    print(e)

./make_file/(최종)_서울열선_광진도로.csv 덮어쓰기 완료.
